In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ijson scikit-learn joblib evaluate


In [3]:
from pathlib import Path

# === Recipe1M layer1.json の場所（Google Drive）===
LAYER1_JSON = Path("/content/drive/MyDrive/recipe1m/layer1.json")

# === 索引キャッシュ（初回重いので Drive 上に保存推奨）===
KB_CACHE = Path("/content/drive/MyDrive/recipe1m_cache/kb_cards_index.joblib")

# === PizzaCommonSense の評価CSV ===
# ここはあなたの既存フローに合わせてください
DATA_DIR = Path("/content/drive/MyDrive/pizza")      # 例
CSV_PATH = DATA_DIR / "pizza_commonsense_test.csv"   # 例：評価用

# === CSV列名：あなたのCSVに合わせて変更 ===
COL_ID    = "id"
COL_INSTR = "instructions"   # モデルへの入力（1行の手順/指示文など）
COL_REF   = "output"         # 正解ラベル

# === Lookup の上位件数 ===
TOP_K = 5

print("layer1.json exists:", LAYER1_JSON.exists())
print("CSV exists:", CSV_PATH.exists())
from pathlib import Path

# === Recipe1M layer1.json の場所（Google Drive）===
LAYER1_JSON = Path("/content/drive/MyDrive/recipe1m/layer1.json")

# === 索引キャッシュ（初回重いので Drive 上に保存推奨）===
KB_CACHE = Path("/content/drive/MyDrive/recipe1m_cache/kb_cards_index.joblib")

# === PizzaCommonSense の評価CSV ===
# ここはあなたの既存フローに合わせてください
DATA_DIR = Path("/content/drive/MyDrive/Completed_CSV")      # 例
CSV_PATH = DATA_DIR / "react_eval_gpt41.csv"   # 例：評価用

# === CSV列名：あなたのCSVに合わせて変更 ===
COL_ID    = "None"
COL_INSTR = "question"   # モデルへの入力（1行の手順/指示文など）
COL_REF   = "ground_truth"         # 正解ラベル

# === Lookup の上位件数 ===
TOP_K = 5

print("layer1.json exists:", LAYER1_JSON.exists())
print("CSV exists:", CSV_PATH.exists())


layer1.json exists: True
CSV exists: False
layer1.json exists: True
CSV exists: True


In [4]:
import pandas as pd

df = pd.read_csv(CSV_PATH).fillna("")
print(df.head(2))
print("Rows:", len(df))
assert COL_INSTR in df.columns and COL_REF in df.columns


                                            question  \
0                      preheat oven to 500 degrees .   
1  put the pizza_shell or possibly focaccia onto ...   

                         react_answer ground_truth  \
0                                                    
1  pizza_shell_on_pan_or_cookie_sheet  pizza_shell   

                                              trace  
0                                  Final Answer: NA  
1  Final Answer: pizza_shell_on_pan_or_cookie_sheet  
Rows: 8841


In [5]:
!pip -q install evaluate bert_score rouge-score

In [6]:
import re, numpy as np, pandas as pd, evaluate, bert_score

df = pd.read_csv(CSV_PATH).fillna("")

rouge  = evaluate.load("rouge")

def ema(preds, refs):
    return np.mean([p == r for p, r in zip(preds, refs)])

def rouge_l(preds, refs):
    return rouge.compute(predictions=preds, references=refs)["rougeL"]

def _to_set(txt):
    return {t.strip() for t in re.split(r"[;,]", txt.lower()) if t.strip()}

def set_f1(preds, refs):
    scores = []
    for p, r in zip(preds, refs):
        P, R = _to_set(p), _to_set(r)
        if not P and not R: scores.append(1); continue
        if not P or not R:  scores.append(0); continue
        inter = len(P & R)
        prec  = inter / len(P);  rec = inter / len(R)
        scores.append(0 if prec+rec == 0 else 2*prec*rec/(prec+rec))
    return np.mean(scores)

BLACK = {"oven","bowl","pan","tray","sheet","mixer",
         "fork","knife","spoon","plate","°f","°c",
         "minutes","minute","timer","preheat"}

def non_food(preds):
    return np.mean([
        1 if any(tok in BLACK for tok in re.findall(r"\w+", p.lower())) else 0
        for p in preds
    ])

def bert_score_f1(preds, refs):
    preds = [p or "empty" for p in preds]
    refs  = [r or "empty" for r in refs]
    return bert_score.score(preds, refs, lang="en", verbose=False)[2].mean().item()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# import ijson, re

# def iter_layer1(path: Path):
#     """layer1.json はレシピ辞書の巨大配列なので1件ずつストリーミングで読む"""
#     with open(path, "rb") as f:
#         for rec in ijson.items(f, "item"):
#             yield rec

# def normalize_txt(s: str) -> str:
#     s = s.strip()
#     s = re.sub(r"\s+", " ", s)
#     return s

# def build_cards_from_layer1(path: Path, limit: int | None = None):
#     """
#     returns:
#       cards:  list[dict] … {id, title, step_idx, text}
#       corpus: list[str]  … 検索用テキスト（= step text）
#     """
#     cards, corpus = [], []
#     n = 0
#     for rec in iter_layer1(path):
#         rid = rec.get("id", "")
#         title = rec.get("title", "")
#         instrs = rec.get("instructions", []) or []
#         for i, it in enumerate(instrs):
#             step = normalize_txt(it.get("text", ""))
#             if not step:
#                 continue
#             cards.append({"id": rid, "title": title, "step_idx": i, "text": step})
#             corpus.append(step)
#         n += 1
#         if limit and n >= limit:
#             break
#     return cards, corpus

# # スモークテスト（重い時は limit=2000 で様子を見る）
# _cards, _corpus = build_cards_from_layer1(LAYER1_JSON, limit=2000)
# len(_cards), _cards[0]


In [8]:
from tqdm.auto import tqdm
import ijson, time, re

def count_layer1_recipes(path):
    cnt = 0
    with open(path, "rb") as f:
        for _ in ijson.items(f, "item"):
            cnt += 1
    return cnt

def normalize_txt(s: str) -> str:
    s = re.sub(r"\s+", " ", s.strip())
    return s

def build_cards_from_layer1(path, limit=None, total=None, report_every=1000):
    """
    returns:
      cards:  list[dict] … {id, title, step_idx, text}
      corpus: list[str]  … 検索用テキスト（step）
    """
    cards, corpus = [], []
    n_rec, n_steps = 0, 0
    t0 = time.perf_counter()
    pbar = tqdm(total=total, desc="Reading layer1") if total else tqdm(desc="Reading layer1")

    with open(path, "rb") as f:
        for rec in ijson.items(f, "item"):
            rid = rec.get("id", "")
            title = rec.get("title", "")
            instrs = rec.get("instructions", []) or []
            for i, it in enumerate(instrs):
                step = normalize_txt(it.get("text", ""))
                if step:
                    cards.append({"id": rid, "title": title, "step_idx": i, "text": step})
                    corpus.append(step)
                    n_steps += 1
            n_rec += 1
            pbar.update(1)
            if report_every and n_rec % report_every == 0:
                elapsed = time.perf_counter() - t0
                pbar.set_postfix(recipes=n_rec, steps=n_steps, sec=int(elapsed))
            if limit and n_rec >= limit:
                break
    pbar.close()
    print(f"[build] recipes={n_rec:,}, steps={n_steps:,}, time={time.perf_counter()-t0:.1f}s")
    return cards, corpus


In [9]:
# 事前カウント（1回目の高速スキャン）
total_recipes = count_layer1_recipes(LAYER1_JSON)

# 実抽出（2回目、進捗バーつき）
cards, corpus = build_cards_from_layer1(LAYER1_JSON, limit=None, total=total_recipes)


Reading layer1:   0%|          | 0/1029720 [00:00<?, ?it/s]

[build] recipes=1,029,720, steps=10,767,598, time=71.3s


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib, time

if KB_CACHE.exists():
    vec, X_cards, cards = joblib.load(KB_CACHE)
    print("[cache] loaded:", KB_CACHE)
else:
    # 上で作った進捗つき関数でコーパス生成
    total_recipes = count_layer1_recipes(LAYER1_JSON)   # 省略可
    cards, corpus = build_cards_from_layer1(LAYER1_JSON, limit=None, total=total_recipes)

    print(f"[tfidf] building on {len(corpus):,} steps ...")
    t0 = time.perf_counter()
    vec = TfidfVectorizer(min_df=3, ngram_range=(1,2))
    X_cards = vec.fit_transform(corpus)
    print(f"[tfidf] done: shape={X_cards.shape}, time={time.perf_counter()-t0:.1f}s")

    KB_CACHE.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump((vec, X_cards, cards), KB_CACHE)
    print("[cache] saved:", KB_CACHE)

len(cards), X_cards.shape



Reading layer1:   0%|          | 0/1029720 [00:00<?, ?it/s]

[build] recipes=1,029,720, steps=10,767,598, time=73.0s
[tfidf] building on 10,767,598 steps ...
[tfidf] done: shape=(10767598, 849111), time=182.9s
[cache] saved: /content/drive/MyDrive/recipe1m_cache/kb_cards_index.joblib


(10767598, (10767598, 849111))

In [14]:
def search_cards(query: str, k: int = TOP_K):
    qv = vec.transform([query])
    sims = cosine_similarity(qv, X_cards).ravel()
    idx = sims.argsort()[::-1][:k]
    results = []
    for j in idx:
        c = cards[j]
        results.append({
            "score": float(sims[j]),
            "text": c["text"],
            "title": c["title"],
            "id": c["id"],
            "step_idx": c["step_idx"],
        })
    return results

def format_observation(results):
    # Observation に貼る短い文字列
    lines = []
    for r in results:
        tag = f"{r['id']}#step{r['step_idx']}"
        lines.append(f"[{tag}] {r['text']} (title: {r['title']})")
    return "\n".join(lines)

def TOOL_LOOKUP(query: str, k: int = TOP_K) -> str:
    """ReAct の Action: Lookup(\"...\") から呼ばれる想定。Observation 文字列を返す。"""
    hits = search_cards(query, k=k)
    return format_observation(hits)


In [15]:
preds = df["react_answer"]
refs  = df["ground_truth"]

metrics = {
    "EMA"          : ema(preds, refs),
    "RougeL"       : rouge_l(preds, refs),
    "SetF1"        : set_f1(preds, refs),
    "NonFood"      : non_food(preds),
    "BERTScore_F1" : bert_score_f1(preds, refs),
}

metrics["StrictScore"] = (
      0.70 * metrics["BERTScore_F1"]
    + 0.15 * metrics["SetF1"]
    + 0.10 * metrics["RougeL"]
    - 0.05 * metrics["NonFood"]
)

print("\n====== Evaluation Summary ======")
for k, v in metrics.items():
    print(f"{k:<12}: {v:.3f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



====== Evaluation Summary ======
EMA         : 0.082
RougeL      : 0.237
SetF1       : 0.082
NonFood     : 0.000
BERTScore_F1: 0.849
StrictScore : 0.630


In [16]:
OUT_CSV = CSV_PATH.with_name("react_eval_gpt41_evaluated2.csv")
out = df.copy()
out.to_csv(OUT_CSV, index=False)
OUT_CSV


PosixPath('/content/drive/MyDrive/Completed_CSV/react_eval_gpt41_evaluated2.csv')